<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Мешок-слов" data-toc-modified-id="Мешок-слов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Мешок слов</a></span></li><li><span><a href="#Построение-моделей-с-применением-BERT" data-toc-modified-id="Построение-моделей-с-применением-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Построение моделей с применением BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install pymorphy2
!pip install pytorch_pretrained_bert
!pip install catboost --no-cache-dir
!pip install lightgbm
!pip install transformers
!pip install pymystem3

Сделаем всё как обычно. Доустановим неустановленное, импортируем библиотеки, загрузим датасет...

In [2]:
import pandas as pd
import numpy as np

import pymorphy2
from pymystem3 import Mystem
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
stopwords = set(nltk_stopwords.words('english'))

import torch
import transformers as ppb
from pytorch_pretrained_bert import BertTokenizer
from tqdm import notebook

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Uglev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Uglev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Uglev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv('toxic_comments.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
display(df['toxic'].value_counts())
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

0    143346
1     16225
Name: toxic, dtype: int64

8.834884437596301

Наблюдается дисбаланс классов. Это наблюдение даст нам значение accuracy для оценки адекватности моделей: она должна быть больше 90%.

В столбце text приведем все сообщения к нижнему регистру. Создадим копию.

In [6]:
df.text = df.text.str.lower()

In [7]:
df_tf_idf = df.copy()

## Обучение

### TF-IDF

Лемматизируем текст.

In [8]:
tokenizer_tf_idf = RegexpTokenizer(r'\w{2,}')
def clean_stemm(text):
    """Функция, отвечающая за лемматизацию слов корпуса
    """
    #new_words = nltk.word_tokenize(text)
    #new_words = tokenizer_tf_idf.tokenize(text)
    # Инициализация лемматизатора
    lemmatizer = WordNetLemmatizer()
    # Лемматизация корпуса
    return " ".join([lemmatizer.lemmatize(w,"n") for w in nltk.word_tokenize(text)])

In [9]:
%%time
df_tf_idf['stem_text'] = df_tf_idf['text'].apply(clean_stemm)

Wall time: 1min 38s


Разделим датафрейм на признак и на целевой признак, далее разделим их на обучающую и тестовую выборки. Выведем информацию о размерах выборок, чтобы убедиться в правильной разбивке:

In [11]:
%%time
corpus_tf_idf = df_tf_idf['stem_text'].values.astype('U')
target_tf_idf = df_tf_idf['toxic']

train_features_tf_idf, test_features_tf_idf, train_target_tf_idf, test_target_tf_idf = train_test_split(
corpus_tf_idf, target_tf_idf, test_size=0.2, random_state=42)

print('Количество строк в обучающей выборке:', train_features_tf_idf.shape[0])
print('Количество строк в тестовой выборке:', test_features_tf_idf.shape[0])

Количество строк в обучающей выборке: 127656
Количество строк в тестовой выборке: 31915
Wall time: 2.34 s


Применим стоп-слова и посмотрим на размер.

In [12]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(train_features_tf_idf)
#tf_idf_test = count_tf_idf.transform(test_features_tf_idf)

И применим логистическую регрессию.

In [13]:
%%time

model_lr_tf_idf = LogisticRegression(random_state=12345, solver='lbfgs', class_weight = 'balanced')
model_lr_tf_idf.fit(tf_idf_train, train_target_tf_idf)
predicted_lr_tf_idf = model_lr_tf_idf.predict(tf_idf_test)
accuracy_lr_tf_idf = accuracy_score(test_target_tf_idf, predicted_lr_tf_idf)
f1_score_lr_tf_idf = f1_score(test_target_tf_idf, predicted_lr_tf_idf)

probabilities_test_lr_tf_idf = model_lr_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_lr_tf_idf = probabilities_test_lr_tf_idf[:, 1]
auc_roc_lr_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_lr_tf_idf)

print('Логистическая регрессия:')
print('accuracy:', accuracy_lr_tf_idf.round(decimals=3))
print('f1_score:', f1_score_lr_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_lr_tf_idf.round(decimals=3))

C:\Users\Uglev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Found input variables with inconsistent numbers of samples: [31915, 127656]

Обучим модель catBoost:

In [ ]:
%%time
model_cat_tf_idf = CatBoostClassifier(iterations=1000,
                                      random_seed = 42,
                                      metric_period = 50,
                                      eval_metric = 'F1',
                                      learning_rate = 0.5,
                                      early_stopping_rounds = 50, 
                                      #task_type="GPU", 
                                      #devices='0:1'
                                     )
model_cat_tf_idf.fit(tf_idf_train, train_target_tf_idf, verbose=200)
predicted_cat_tf_idf = model_cat_tf_idf.predict(tf_idf_test)
accuracy_cat_tf_idf = accuracy_score(test_target_tf_idf, predicted_cat_tf_idf)
f1_score_cat_tf_idf = f1_score(test_target_tf_idf, predicted_cat_tf_idf)

probabilities_test_cat_tf_idf = model_cat_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_cat_tf_idf = probabilities_test_cat_tf_idf[:, 1]
auc_roc_cat_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_cat_tf_idf)

print('CatBoost:')
print('accuracy:', accuracy_cat_tf_idf.round(decimals=3))
print('f1_score:', f1_score_cat_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_cat_tf_idf.round(decimals=3))

0:	learn: 0.4910376	total: 820ms	remaining: 13m 39s
200:	learn: 0.8113133	total: 1m 57s	remaining: 7m 47s
400:	learn: 0.8371017	total: 3m 53s	remaining: 5m 48s
600:	learn: 0.8489900	total: 5m 49s	remaining: 3m 51s


Теперь обучим LightGBM на стандартных настройках гиперпараметров:

In [ ]:
%%time
model_lgb_tf_idf = lgb.LGBMClassifier()
model_lgb_tf_idf.fit(tf_idf_train, train_target_tf_idf)
predicted_lgb_tf_idf = model_lgb_tf_idf.predict(tf_idf_test)
accuracy_lgb_tf_idf = accuracy_score(test_target_tf_idf, predicted_lgb_tf_idf)
f1_score_lgb_tf_idf = f1_score(test_target_tf_idf, predicted_lgb_tf_idf)

probabilities_test_lgb_tf_idf = model_lgb_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_lgb_tf_idf = probabilities_test_lgb_tf_idf[:, 1]
auc_roc_lgb_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_lgb_tf_idf)

print('LightGBM:')
print('accuracy:', accuracy_lgb_tf_idf.round(decimals=3))
print('f1_score:', f1_score_lgb_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_lgb_tf_idf.round(decimals=3))

### Построение моделей с применением BERT

In [ ]:
df_bert_ones = df[df['toxic']==1].sample(1000).reset_index(drop=True)
df_bert_zeros = df[df['toxic']==0].sample(1000).reset_index(drop=True)
df_bert = pd.concat([df_bert_ones] + [df_bert_zeros]).reset_index(drop=True)
df_bert = shuffle(df_bert, random_state=12345).reset_index(drop=True)

df_bert.toxic.value_counts()

Создадим токенайзер.

In [ ]:
tokenizer = ppb.BertTokenizer.from_pretrained('bert-base-uncased')

Токенизируем текст каждого твита.

In [ ]:
tokenized = df_bert['text'].apply((lambda x: tokenizer.encode(x, max_length=512, add_special_tokens=True)) )

Инициализируем модель из файла:

In [ ]:
config = ppb.BertConfig.from_pretrained('bert-base-uncased')
model = ppb.BertModel.from_pretrained('bert-base-uncased', config = config)

Заполними нулями каждый массив токенов до длины максимального массива, чтобы получить матрицу одной длины. Наложим маску на значимые токены. Нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга.

In [ ]:
len_list = tokenized.apply(lambda x: len(x))
max_len = max(len_list)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Сформируем векторы текстов:

In [ ]:
batch_size = 50
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask_batch)

        embeddings.append(last_hidden_states[0][:,0,:].numpy())

Преобразуем список батчей эмбеддингов в numpy-матрицу, в другую матрицу запишем значения целевого признака, и разделим матрицы на обучающую и тестовую выборки:

In [ ]:
features = np.concatenate(embeddings)
labels = df_bert['toxic']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            test_size=0.25, random_state=42)

Обучаем модели.

In [ ]:
%%time
model_lr = LogisticRegression(solver = 'lbfgs', class_weight = 'balanced', max_iter = 1000)
model_lr.fit(train_features, train_labels)
predicted_lr = model_lr.predict(test_features)
accuracy_lr = accuracy_score(test_labels, predicted_lr)
f1_score_lr = f1_score(test_labels, predicted_lr)

probabilities_test_lr = model_lr.predict_proba(test_features)
probabilities_one_test_lr = probabilities_test_lr[:, 1]
auc_roc_lr = roc_auc_score(test_labels, probabilities_one_test_lr)

print('Логистическая регрессия:')
print('accuracy:', accuracy_lr.round(decimals=3))
print('f1_score:', f1_score_lr.round(decimals=3))
print('auc_roc:', auc_roc_lr.round(decimals=3))

In [ ]:
%%time
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(train_features, train_labels)
predicted_lgb = model_lgb.predict(test_features)
accuracy_lgb = accuracy_score(test_labels, predicted_lgb)
f1_score_lgb = f1_score(test_labels, predicted_lgb)

probabilities_test_lgb = model_lgb.predict_proba(test_features)
probabilities_one_test_lgb = probabilities_test_lgb[:, 1]
auc_roc_lgb = roc_auc_score(test_labels, probabilities_one_test_lgb)

print('LightGBM:')
print('accuracy:', accuracy_lgb.round(decimals=3))
print('f1_score:', f1_score_lgb.round(decimals=3))
print('auc_roc:', auc_roc_lgb.round(decimals=3))

In [ ]:
%%time
model_cat = CatBoostClassifier(iterations=1000,
                               random_seed = 42,
                               metric_period = 50,
                               eval_metric = 'F1',
                               learning_rate = 0.5,
                               early_stopping_rounds = 50, 
                               #task_type="GPU", 
                               #devices='0:1'
                              )

model_cat.fit(train_features, train_labels, verbose=200)
predicted_cat = model_cat.predict(test_features)
accuracy_cat = accuracy_score(test_labels, predicted_cat)
f1_score_cat = f1_score(test_labels, predicted_cat)

probabilities_test_cat = model_cat.predict_proba(test_features)
probabilities_one_test_cat = probabilities_test_cat[:, 1]
auc_roc_cat = roc_auc_score(test_labels, probabilities_one_test_cat)

print('CatBoost:')
print('accuracy:', accuracy_cat.round(decimals=3))
print('f1_score:', f1_score_cat.round(decimals=3))
print('auc_roc:', auc_roc_cat.round(decimals=3))

## Выводы

Все использованные модели сильно превысили необходимое значение метрики F1=0.75. С применением TF-IDF лучшей моделью по значению метрики F1 является CatBoost (F1=0.78). С применением BERT лучшей моделью по значению метрики F1 является логистическая регрессия (F1=0.865). У нее также самые большие значения Accuracy и ROC-AUC и наименьшее время обучения (< 1 сек) по сравнению с CatBoost и LightGBM.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны